# 👥 Customer Segmentation

This notebook performs customer segmentation using clustering algorithms.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import plotly.express as px
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

DATA_PATH = Path('../DataSource')
OUTPUT_PATH = Path('../outputs/predictions')

## 1. Data Preparation

In [ ]:
# Load data
sales = pd.read_csv(DATA_PATH / 'Candy_Sales.csv')
sales['Order Date'] = pd.to_datetime(sales['Order Date'])

print(f"Total records: {len(sales):,}")
print(f"Unique customers: {sales['Customer ID'].nunique():,}")

In [ ]:
# Create RFM features
reference_date = sales['Order Date'].max() + pd.Timedelta(days=1)

rfm = sales.groupby('Customer ID').agg({
    'Order Date': lambda x: (reference_date - x.max()).days,  # Recency
    'Order ID': 'nunique',  # Frequency
    'Sales': 'sum'  # Monetary
}).reset_index()

rfm.columns = ['Customer ID', 'Recency', 'Frequency', 'Monetary']

# Additional features
customer_features = sales.groupby('Customer ID').agg({
    'Units': 'sum',
    'Gross Profit': 'sum',
    'Product ID': 'nunique',
    'Division': 'nunique'
}).reset_index()
customer_features.columns = ['Customer ID', 'Total_Units', 'Total_Profit', 'Products_Bought', 'Divisions_Bought']

# Merge
customers = rfm.merge(customer_features, on='Customer ID')

# Calculate derived features
customers['Avg_Order_Value'] = customers['Monetary'] / customers['Frequency']
customers['Avg_Units_Per_Order'] = customers['Total_Units'] / customers['Frequency']
customers['Profit_Margin'] = customers['Total_Profit'] / customers['Monetary']

print(customers.head())

In [ ]:
# Summary statistics
print("\nCustomer Feature Statistics:")
print(customers.describe())

## 2. Feature Scaling

In [ ]:
# Select features for clustering
cluster_features = ['Recency', 'Frequency', 'Monetary', 'Avg_Order_Value', 'Products_Bought']
X = customers[cluster_features]

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"Features for clustering: {cluster_features}")
print(f"Shape: {X_scaled.shape}")

## 3. Optimal Clusters (Elbow Method)

In [ ]:
# Elbow method
inertias = []
K_range = range(2, 10)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(K_range, inertias, 'bo-', linewidth=2)
ax.set_xlabel('Number of Clusters (K)')
ax.set_ylabel('Inertia')
ax.set_title('Elbow Method for Optimal K')
ax.axvline(x=4, color='red', linestyle='--', alpha=0.7, label='Selected K=4')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

## 4. K-Means Clustering

In [ ]:
# Apply K-Means with K=4
n_clusters = 4
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
customers['Cluster'] = kmeans.fit_predict(X_scaled)

print(f"Cluster distribution:")
print(customers['Cluster'].value_counts().sort_index())

In [ ]:
# Cluster profiles
cluster_profile = customers.groupby('Cluster')[cluster_features + ['Total_Profit']].mean()
cluster_profile['Count'] = customers.groupby('Cluster').size()
cluster_profile['% of Customers'] = (cluster_profile['Count'] / len(customers) * 100).round(1)

print("\nCluster Profiles:")
print(cluster_profile.round(2))

In [ ]:
# Assign segment names based on profile
segment_names = {
    0: 'Champions',      # High value, frequent
    1: 'At Risk',        # Low recency, low frequency
    2: 'Potential',      # Medium everything
    3: 'New Customers'   # High recency, low frequency
}

# Analyze and assign based on actual cluster characteristics
cluster_summary = cluster_profile[['Recency', 'Frequency', 'Monetary']].copy()

# Assign names based on RFM patterns
def assign_segment(row):
    if row['Frequency'] > cluster_profile['Frequency'].median():
        if row['Monetary'] > cluster_profile['Monetary'].median():
            return 'Champions'
        else:
            return 'Loyal'
    else:
        if row['Recency'] < cluster_profile['Recency'].median():
            return 'Potential'
        else:
            return 'At Risk'

cluster_labels = cluster_profile.apply(assign_segment, axis=1)
customers['Segment'] = customers['Cluster'].map(cluster_labels)

print("\nSegment Distribution:")
print(customers['Segment'].value_counts())

## 5. Visualization

In [ ]:
# PCA for 2D visualization
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

customers['PCA1'] = X_pca[:, 0]
customers['PCA2'] = X_pca[:, 1]

fig = px.scatter(customers, x='PCA1', y='PCA2', color='Segment',
                 title='Customer Segments (PCA Visualization)',
                 hover_data=['Customer ID', 'Monetary', 'Frequency'])
fig.show()

In [ ]:
# RFM Scatter
fig = px.scatter(customers, x='Frequency', y='Monetary', color='Segment',
                 size='Total_Profit', title='Customer Segments: Frequency vs Monetary',
                 labels={'Frequency': 'Order Frequency', 'Monetary': 'Total Spend ($)'})
fig.show()

In [ ]:
# Segment comparison - Box plots
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

metrics = ['Monetary', 'Frequency', 'Recency', 'Avg_Order_Value']
for ax, metric in zip(axes.flat, metrics):
    customers.boxplot(column=metric, by='Segment', ax=ax)
    ax.set_title(metric)
    ax.set_xlabel('Segment')

plt.suptitle('Segment Comparison', fontsize=14)
plt.tight_layout()
plt.show()

## 6. Export Results

In [ ]:
# Save customer segments
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)
export_cols = ['Customer ID', 'Segment', 'Cluster', 'Recency', 'Frequency', 'Monetary', 
               'Avg_Order_Value', 'Products_Bought', 'Total_Profit']
customers[export_cols].to_csv(OUTPUT_PATH / 'customer_segments.csv', index=False)

print(f"Customer segments saved to: {OUTPUT_PATH / 'customer_segments.csv'}")

## 7. Segment Summary & Recommendations

In [ ]:
# Final summary
segment_summary = customers.groupby('Segment').agg({
    'Customer ID': 'count',
    'Monetary': ['sum', 'mean'],
    'Frequency': 'mean',
    'Recency': 'mean',
    'Total_Profit': 'sum'
}).round(2)

segment_summary.columns = ['Customers', 'Total Revenue', 'Avg Revenue', 'Avg Frequency', 'Avg Recency', 'Total Profit']
segment_summary['% Revenue'] = (segment_summary['Total Revenue'] / segment_summary['Total Revenue'].sum() * 100).round(1)

print("\n" + "="*70)
print("CUSTOMER SEGMENTATION SUMMARY")
print("="*70)
print(segment_summary)

print("\n" + "="*70)
print("SEGMENT RECOMMENDATIONS")
print("="*70)
recommendations = {
    'Champions': '🏆 VIP treatment, exclusive offers, loyalty rewards',
    'Loyal': '💎 Upsell premium products, referral programs',
    'Potential': '📈 Targeted promotions, engagement campaigns',
    'At Risk': '⚠️ Win-back campaigns, special discounts to re-engage'
}

for segment, recommendation in recommendations.items():
    if segment in customers['Segment'].values:
        print(f"\n{segment}: {recommendation}")